In [1]:
import whisper
import torch
from transformers import pipeline
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import re


c:\Users\famel1x\Desktop\MS_cutter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Функция для извлечения аудио из видео
def extract_audio(video_path):
    video = VideoFileClip(video_path)
    audio_path = video_path.replace(".mp4", ".wav")  # Сохраняем аудио как .wav
    video.audio.write_audiofile(audio_path, codec='pcm_s16le')  # Сохраняем аудио в формате .wav
    return audio_path

In [3]:
# Функция для транскрибации аудио с Whisper
def transcribe_audio(audio_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("small", device=device)
    
    # Транскрибируем аудио
    result = model.transcribe(audio_path, language="ru")
    
    segments = result["segments"]
    
    # Отображаем прогресс по мере обработки сегментов
    transcribed_segments = []
    for segment in tqdm(segments, desc="Transcribing Audio"):
        transcribed_segments.append(segment)
    
    return result["text"], transcribed_segments

In [4]:
# Функция для выбора интересных моментов с использованием модели суммаризации
def select_interesting_segments(text, num_segments):
    device = 0  # Запуск модели на CPU для устранения ошибок
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    
    # Разбиваем текст на блоки длиной не более 1024 символов
    max_block_size = 128
    text_blocks = [text[i:i+max_block_size] for i in range(0, len(text), max_block_size)]
    
    interesting_segments = []
    
    for i, block in enumerate(tqdm(text_blocks, desc="Summarizing Text")):
        if not block.strip():
            print(f"Блок {i} пуст, пропускаем...")
            continue
        
        try:
            if len(block) < 50:
                print(f"Блок {i} слишком короткий, пропускаем...")
                continue
            
            summary = summarizer(block, max_length=150, min_length=30, do_sample=False)
            interesting_segments.append(summary[0]['summary_text'])
        except Exception as e:
            print(f"Ошибка суммаризации блока {i}: {e}")
            print(f"Проблемный блок: {block}")
            continue
    
    return interesting_segments[:num_segments]


In [5]:
# Функция для генерации названия файла из текста
def generate_filename(text):
    clean_text = re.sub(r'\W+', ' ', text).strip()
    filename = "_".join(clean_text.split()[:5])
    return filename[:50]

In [6]:
# Функция для сопоставления интересных сегментов с временными метками и нарезки
def cut_video(video_path, segments, interesting_segments, num_clips=12):
    max_clip_length = 120  # Максимальная длина клипа (в секундах)
    min_clip_length = 40  # Минимальная длина клипа (в секундах)
    
    # Сопоставляем текст с временными метками
    segment_times = []
    for i, segment in enumerate(segments):
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text']
        
        # Проверяем, совпадает ли текст с интересными моментами
        if any(interesting_segment in text for interesting_segment in interesting_segments):
            segment_times.append((start_time, end_time, text))

    # Если нашли интересные моменты, нарезаем видео
    if not segment_times:
        print("Интересные моменты не найдены.")
        return
    
    total_clips = []
    current_clip = []
    clip_count = 0
    
    for i, (start_time, end_time, text) in enumerate(tqdm(segment_times, desc="Processing Clips")):
        duration = end_time - start_time
        current_clip.append((start_time, end_time, text))
        
        # Собираем клипы по длительности
        if len(current_clip) > 1 and (sum([c[1] - c[0] for c in current_clip]) >= min_clip_length):
            if clip_count >= num_clips:
                break
            
            total_clips.append(current_clip)
            current_clip = []
            clip_count += 1
    
    # Нарезка видео клипов
    for i, clip in enumerate(total_clips):
        start_time = clip[0][0]
        end_time = clip[-1][1]
        summary_text = " ".join([seg[2] for seg in clip])
        
        filename = generate_filename(summary_text)
        output_file = f"{filename}_clip_{i}.mp4"
        
        print(f"Сохраняем клип: {output_file} (с {start_time} по {end_time})")
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_file)

In [7]:
video_path = "interview.mp4"
print("Извлечение аудио...")
audio_path = extract_audio(video_path)



Извлечение аудио...
MoviePy - Writing audio in interview.wav


MoviePy - Done.


In [8]:
# 2. Транскрибирование речи с прогрессом
print("Транскрибирование аудио...")
text, segments = transcribe_audio(audio_path)




Транскрибирование аудио...


Transcribing Audio: 100%|██████████| 649/649 [00:00<?, ?it/s]


In [9]:
# 3. Выбор интересных моментов с учётом прогресса
print("Выбор интересных моментов...")
interesting_segments = select_interesting_segments(text, 10)  # Выбираем 10 сегментов текста

Выбор интересных моментов...


Summarizing Text:   2%|▏         | 4/217 [00:12<10:58,  3.09s/it]


KeyboardInterrupt: 

In [17]:
print("Интересные моменты:")
for i, segment in enumerate(interesting_segments):
    print(f"{i+1}. {segment}")

Интересные моменты:
1.  причине ее не    папище,  ‘ обязательно по   “перегент”,  “На  разрыв.” ‘‘’Не тактер,’ “”’ “’,”  ‘Н” ”Н ”“, ”,  ”.
2.  е. припаркома,  ‘нтервь’,   часть разговоров,  ‘На  Не  тепере, т. “Ня такте,”  “” ‘’’ “т.”, “М, ‘, ’”.
3.  пригот  прегант    ‘перент’,   ‘‘’’  “’п’.  ’ ’ ””.” “Н. щ.  шим  райте, т. “”  ”“Љ.’ ”, “т.“, ”. “   ‚”,   .  “ 
4.  подпищитеперь    погнали  регентреть  “не напрактернета  фантете   ‘грене’,   ‘’Н. Коля  танение, ‘Канна’  ’’. ’‘”’: “Кенщ
5.  оперейне   тебе тактер,   “Т.е. щ.  п.  х. реще, т. ‘Н. Н. я  сменил немножко деятельность.’. ‘‘’’   ‘т.S. “”   Не раз,’ ” �
6.  Нет,   такая   перезагруз.   На тей,  “Не’,  “”, ”“,”  ‘’” “, ”, “ ”.
7. The video was posted on YouTube.com. It was created by a user named    ‘Нериальный.’.
8.  Нет. Я пре    генте,  ‘Не’, ‘‘”’ “’”, “Н”. “ ” ’.  “” “I’m sorry,” she said. ‘I don’t know what to say,’ she said, ’cause I’ve never heard of it.’
9.  Изначально я считал это    преграще,  “Не  там мне чта,   “пе

In [18]:
print("Нарезка видео...")
cut_video(video_path, segments, interesting_segments)

Нарезка видео...
Интересные моменты не найдены.


In [19]:
text

' Вписка с папичем, которую мы недавно дропнули, разрыв. И вот если вы по какой-то причине ее не видели, то обязательно посмотрите. Ссылку мы припарковали в описании. Интервью с папани снималась два дня, и часть разговоров не попали в основной выпуск. По вашим просьбам мы собрали самые интересные кусочки изнеизданного и приготовили допник. Запаситесь едой, поставьте большой палец и подпишитесь на канал. А теперь погнали смотреть выпуск. Вася, Коля вписка. Вася, Коля вписка. Вася, Коля вписка. Мне надоело с своими классическими стримами игр пытаться угодить зрителю. Ну и вот, собственно, я сменил немножко деятельность. Т.е. тебе нужна была такая встрязка, перезагруз. Нет, нет, нет, нет. Просто не могу больше стремиться только как раньше. Вот и все. Ну соответственно, тут здесь у тебя новый вариант, это видеоролики на YouTube. Сериальный. И тебе показалось что-то интересным вызовом, так? Ммм. Нет. Я предложили достаточно много денег. Вообще, я ж говорю, я изначально считал эти видеоролик

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загружаем модель и токенизатор
model_name = "bigscience/bloom-3b"  # Вы можете выбрать модель с меньшим или большим количеством параметров
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



# Функция для генерации текста на основе промпта
def generate_text(prompt, max_length=400):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=max_length, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Пример использования
prompt = f"""
Вот текст интервью. Выбери 10 наиболее интересных моментов из текста и процитируй их с объяснением, почему они важны. 
Текст интервью:
Спустя 10 лет стримов, искренний интерес к играм у меня не так часто. То есть местами вот мне нравится что-то стримить, местами не сильно нравится. Ну, чтобы супер сильно нравилось, это получается в последние годы, ну, раз в 2-3 месяца. То есть, раз в 2-3 месяца стримлю на протяжении двух-трех недель какой-то интересную игру, потом опять я стримлю то, что приходится, то, что получается, то, что попросили, то, что популярно нынче. Как пойдет? А ты допускал, что при переезде частья твоей креноидитории, который привыкло воспринимать тебе как достаточно эскетичного человека, разочаруйся в те. 
Выбери моменты:
"""

prompt2 = "Вот текст интервью выбери то , что тебе понравилось и почему это важно. Текст интервью: Спустя 10 лет стримов, искренний интерес к играм у меня не так часто. То есть местами вот мне нравится что-то стримить, местами не сильно нравится. Ну, чтобы супер сильно нравилось, это получается в последние годы, ну, раз в 2-3 месяца. То есть, раз в 2-3 месяца стримлю на протяжении двух-трех недель какой-то интересную игру, потом опять я стримлю то, что приходится, то, что получается, то, что попросили, то, что популярно нынче. Как пойдет? А ты допускал, что при переезде частья твоей креноидитории, который привыкло воспринимать тебе как достаточно эскетичного человека, разочаруйся в те. "

promt3 = "Привет"

interesting_moments = generate_text(promt3)
print(interesting_moments)

Приветствуйте, я хочу сделать так, чтобы при нажатии на кнопку, всё было выглядеть как на этом примере:
http://codepen.io/anon/pen/PMxZp
Например, если нажимаю на "Создать", то должно выводиться список созданных записи, и еще ссылка на сайт, который содержит собственно следующую страницу, состоящей из текста "Пользователь" и "Запись". На этой строчке долгосрочный слой, на котором будет выведен только тот тэг, что я написала в коде. При наведении на тему, долгоослое долзать выезжать, а с сервера должина выполняться действие, которая будем вызвать при смене темы.
Спасибо за помощь!

A:

Попробуем с помодью jQuery:

$(document).ready(function() {
  $('.btn').click( function(){
    $.ajax({
      url: './create.php',
      type: 'POST',
      data: {
        name: $().val()
      },
      success: function(data) {
       $()
       }
    });
  })
})

$().ready(() => {
   $(()
  ).on('click', 'span.name', function () {
    alert($()[0].innerText) // вывод того, как


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Загрузка модели и токенизатора
model_name = "EleutherAI/gpt-neo-1.3B"  # Или другая модель (например, "EleutherAI/gpt-neo-2.7B")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to('cuda')  # Перемещаем модель на GPU

# Функция для разделения текста на части по предложениям
def split_text_into_chunks(text, max_chunk_size=256+128):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# Функция для генерации интересных моментов из текста
def generate_interesting_moments(text_chunk, max_length=512):
    inputs = tokenizer(text_chunk, return_tensors="pt").to('cuda')  # Перемещаем данные на видеокарту
    outputs = model.generate(inputs['input_ids'], max_new_tokens=max_length, no_repeat_ngram_size=2)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Основная функция для обработки большого текста
def process_large_text(text, num_moments=10, chunk_size=512):
    chunks = split_text_into_chunks(text, chunk_size)
    all_interesting_moments = []

    # Обрабатываем каждый фрагмент текста
    for chunk in chunks:
        interesting_moment = generate_interesting_moments(chunk)
        all_interesting_moments.append(interesting_moment)

    # Собираем итоговые моменты, ограничивая их количество
    selected_moments = all_interesting_moments[:num_moments]
    
    return selected_moments

# Пример большого текста на русском языке
large_text = text

# Запуск обработки текста
interesting_moments = process_large_text(large_text, num_moments=10)

# Вывод интересных моментов
for i, moment in enumerate(interesting_moments):
    print(f"Интересный момент {i+1}: {moment}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 